## Install neccessary packages

In [ ]:
# Python 3.11.13
!python --version

In [ ]:
# %pip install -r ./extract_text/requirements.txt
# %pip install -r ./text_normalizer_vietnamese/requirements.txt
# %pip install pymupdf
# %pip install matplotlib pandas
# %pip install -r ./sentence_alignment/requirements.txt

In [ ]:
%pip install -r requirements.txt
%pip install -r ./quocngu_normalizer/requirements.txt

In [ ]:
%pip install matplotlib

In [ ]:
%pip install sentence_transformers

In [ ]:
%pip install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re
from pathlib import Path
from typing import Literal

from pdf_extractor import SinoNomPDFExtractor, QuocNguPDFExtractor
from quocngu_preprocessing import quocngu_preprocessing
from sinonom_preprocessing import sinonom_preprocessing

from utils import extract_quocngu_section_number, extract_sino_section_number, convert_list_to_dict
from alignment import align_section

In [ ]:
PAGE_BREAK = "\f\n"
PARAGRAPH_BREAK = "\n\n"
SENTENCE_BREAK = "\n"
CHINESE_CHAPTER_TEMPLATE = r"^第[一二三四五六七八九十百千零〇○]+[回囘]"
VIETNAMESE_CHAPTER_TEMPLATE = r"^HỒI THỨ(?: [\wÀ-Ỵ]+)+$"

QN_input_file = "./data/source/Book-Tay_Du_Ky-Viet.pdf"
SN_input_file = "./data/source/Book-Tay_Du_Ky-Trung.pdf"

QN_raw_folder = "./data/raw/QuocNgu"
SN_raw_folder = "./data/raw/SinoNom"

QN_preprocessed_folder = "./data/preprocessed/QuocNgu"
SN_preprocessed_folder = "./data/preprocessed/SinoNom"

xml_output_folder = "./data/output/"

In [ ]:
%load_ext autoreload
%autoreload 2

sino_extractor = SinoNomPDFExtractor(SN_input_file, 1, 10)
quocngu_extractor = QuocNguPDFExtractor(QN_input_file, 48, 50)

# =============================================
# Get raw text
sino_sections = sino_extractor.get_splitted_sections(CHINESE_CHAPTER_TEMPLATE)
quocngu_sections = quocngu_extractor.get_splitted_sections(VIETNAMESE_CHAPTER_TEMPLATE)

# save_quocngu_sections(Path(QN_raw_folder), quocngu_sections, True)

# print(quocngu_sections[0])

# =============================================
# Preprocessing
sinonom_sentence_dict = {}
for section in sino_sections:
    section_num = extract_sino_section_number(section)
    section = sinonom_preprocessing(section)
    sinonom_sentence_dict[section_num] = convert_list_to_dict(section)
# print(sinonom_sentence_dict)

quocngu_sentence_dict = {}
for section in quocngu_sections:
    section_num = extract_quocngu_section_number(section)
    section = quocngu_preprocessing(section)
    quocngu_sentence_dict[section_num] = section
# print(quocngu_sentence_dict)

# =============================================
# Sentence Alignment
for key in sinonom_sentence_dict:
    if key in quocngu_sentence_dict.keys():
        results = align_section(sinonom_sentence_dict[key], quocngu_sentence_dict[key])
        break

# =============================================
# Save output

## 1. Lấy dữ liệu văn bản từ tệp PDF

In [ ]:
# VĂN BẢN CHỮ QUỐC NGỮ
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Viet.pdf -l viet -o ./data/raw_text/vietnamese -fp 48 -np 2 --rule_path ./pdf_to_text/removal_rules_vietnamese.json --use_rule
# Trang chứa nội dung: 48 -> 3415

In [ ]:
# VĂN BẢN CHỮ HÁN
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Trung.pdf -l trung -o ./data/raw_text/chinese -fp 1 -np 2 --rule_path ./pdf_to_text/removal_rules_chinese.json --use_rule --use_page_split
# !python ./extract_text/extract_text_from_pdf.py
# Trang chứa nội dung: 1 -> 542

## 2. Chuẩn hóa dữ liệu văn bản 

### 2.1 Chuẩn hóa văn bản chữ Quốc ngữ

In [ ]:
# !python ./text_normalizer_vietnamese/main.py ./data/raw_text/vietnamese/ ./data/cleaned_text/vietnamese

### 2.2 Chuẩn hóa văn bản chữ Hán

In [ ]:
# !python ./text_normalizer_chinese/main.py -i ./data/raw_text/chinese/ -o ./data/cleaned_text/chinese

In [ ]:
# !python ./sentence_alignment/sentence_alignment.py

In [ ]:
# %pip install pdfplumber

In [ ]:
# %pip install "numpy<=2.0.0" --force-reinstall

In [1]:
%load_ext autoreload
%reload_ext autoreload

!python main.py

2025-06-24 22:09:39,809 - ParallelCorpusGenerator - INFO - Successfully extracted text from page 1 to 10 from SinoNom PDF text.
2025-06-24 22:09:39,811 - ParallelCorpusGenerator - INFO - Successfully extracted text from page 48 to 97 from Vietnamese PDF text.
Source language: Chinese, Number of sentences: 331
Target language: Vietnamese, Number of sentences: 383
Embedding source and target text using LaBSE ...
Performing first-step alignment ...
Performing second-step alignment ...
Finished! Successfully aligning 331 Chinese sentences to 383 Vietnamese sentences

2025-06-24 22:13:14,318 - ParallelCorpusGenerator - INFO - Successfully align SinoNom-Vietnamese text from section 1
第一回靈根育孕源流出心性修持大道生
HỒI THỨ NHẤT Gốc thiêng ấp ủ, nguồn rộng chảy. Tâm tính sửa sang, đạo lớn sinh.

詩曰:混沌未分天地亂，茫茫渺渺無人見。
Có bài thơ rằng: Thuở hoang sơ đất trời chưa tỏ. Chốn mênh mông nào có bóng người.

自從盤古破鴻濛，開闢從茲清濁辨。
Từ khi Bàn Cổ ra đời. Đục trong phân biệt, khác thời hỗn mang.

覆載群生仰至仁，發明萬物皆成善。
Che chở khắp

In [ ]:
import re

# template = r"^第([一二三四五六七八九十百千零〇○]+)[回囘]"
# line = "第一回 靈根育孕源流出 心性修持大道生"

template = r"^HỒI THỨ ([\wÀ-Ỵ]+(?: [\wÀ-Ỵ]+)*)"
line = "HỒI THỨ NHẤT\nGốc thiêng ấp ủ, nguồn rộng chảy\nTâm tính sửa sang, đạo lớn sinh\n"

pattern = re.compile(template)
if pattern.match(line):
    # print(pattern.search(line))
    print(pattern.search(line).group(0))
    print("Match")

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import re
from pathlib import Path
from typing import Literal

from pdf_extractor import SinoNomPDFExtractor, QuocNguPDFExtractor
from quocngu_preprocessing import quocngu_preprocessing
from sinonom_preprocessing import sinonom_preprocessing
from utils import extract_quocngu_section_number, extract_sino_section_number, convert_list_to_dict, save_text
from alignment import align_section

In [ ]:
PAGE_BREAK = "\f\n"
PARAGRAPH_BREAK = "\n\n"
SENTENCE_BREAK = "\n"

CHINESE_CHAPTER_TEMPLATE = r"^第[一二三四五六七八九十百千零〇○]+[回囘]"
VIETNAMESE_CHAPTER_TEMPLATE = r"^HỒI THỨ(?: [\wÀ-Ỵ]+)+"

QN_input_file = "./data/source/Book-Tay_Du_Ky-Viet.pdf"
SN_input_file = "./data/source/Book-Tay_Du_Ky-Trung.pdf"

QN_raw_folder = "./data/raw/QuocNgu"
SN_raw_folder = "./data/raw/SinoNom"

QN_preprocessed_folder = "./data/preprocessed/QuocNgu"
SN_preprocessed_folder = "./data/preprocessed/SinoNom"

xml_output_folder = "./data/output/"

In [ ]:
sino_extractor = SinoNomPDFExtractor(SN_input_file, 1, 10)
quocngu_extractor = QuocNguPDFExtractor(QN_input_file, 48, 50)

# =============================================
# Get raw text
sino_sections = sino_extractor.get_splitted_sections(CHINESE_CHAPTER_TEMPLATE)
quocngu_sections = quocngu_extractor.get_splitted_sections(VIETNAMESE_CHAPTER_TEMPLATE)
print("Get raw text successfully!!!")

# Get section names
quocngu_sect_name_dict = {}
for sect in quocngu_sections:
    match = re.search(VIETNAMESE_CHAPTER_TEMPLATE, sect)
    if not match:
        print("Không tìm thấy tên chương")
        continue 
    sect_name = match.group(0)
    sect_id = extract_quocngu_section_number(sect_name)
    if sect_id and sect_id > 0:
        quocngu_sect_name_dict[str(sect_id)] = sect_name
print("Get section names successfully!!!")
# save_quocngu_sections(Path(QN_raw_folder), quocngu_sections, True)

# Save raw text
raw_sino_folder_path = Path(SN_raw_folder)
raw_sino_folder_path.mkdir(parents=True, exist_ok=True)
for sect in sino_sections:
    match = re.search(VIETNAMESE_CHAPTER_TEMPLATE, sect)



# =============================================
# Preprocessing
# sinonom_sentence_dict = {}  
# for section in sino_sections:
#     section_num = extract_sino_section_number(section)
#     section = sinonom_preprocessing(section)
#     sinonom_sentence_dict[section_num] = convert_list_to_dict(section)
# # print(sinonom_sentence_dict)
# print("Preprocess SinoNom text successfully!!!")

# quocngu_sentence_dict = {}
# for section in quocngu_sections:
#     section_num = extract_quocngu_section_number(section)
#     section = quocngu_preprocessing(section)
#     quocngu_sentence_dict[section_num] = section
# # print(quocngu_sentence_dict)
# print("Preprocess vietnamese text successfully!!!")


# Save preprocessed text

# =============================================
# Sentence Alignment
# aligned_section_list = []
# for key in sinonom_sentence_dict:
#     if key in quocngu_sentence_dict.keys():
#         aligned_section_list.append(align_section(sinonom_sentence_dict[key], quocngu_sentence_dict[key], int(key)))
#         break
# print("Align sentences successfully!!!")

# =============================================
# Save XML output
# from xml_builder.xml_builder import XMLBuilder

# builder = XMLBuilder(
#     file="TDK_001",
#     title="Tây Du Ký",
#     volume="",
#     author="Ngô Thừa Ân",
#     period="",
#     language="Hán-Việt",
#     source="",
# )

# for sect_dict in aligned_section_list:
#     sect_id = sect_dict['section_value']
#     page_list = []
#     page_pair_list = []
#     for page_dict in sect_dict['section_content']:
#         page_id = page_dict['page_value']
#         pair_list = []
#         for sent_dict in page_dict['page_content']:
#             pair_list.append(sent_dict["sentence_content"])
#         page_list.append(str(page_id))
#         page_pair_list.append(pair_list)

#     builder.add_pair_sentence(sect_id=str(sect_id), sect_name=quocngu_sect_name_dict[str(sect_id)], pairs=page_pair_list, paragraphs_lst=page_list)

# builder.save(output_dir = Path(xml_output_folder))
# print("Save XML successfully!!!")
